In [1]:
# default_exp util

In [2]:
%pip install nbdev==1.2.11 | grep -v 'already satisfied'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append('..')
import lib_py.ipython_magic as ipython_magic

# spiral

In [4]:
%%spi

inl test b = assert b "test"


<magic:1d2cddd7f01e4c0c8da18c39fc3f3d5e>


<IPython.core.display.Javascript object>

'/workspaces/tictactoe_spiral/lib_spi/util.spi'

# node

In [5]:
%%node export

export type PickByType<T, Value> = {
    [P in keyof T as T[P] extends Value | undefined ? P : never]: T[P]
}

type NoneEmptyArray = readonly any[] & { 0: any }
type CompareUnionWithArray<P, Q extends NoneEmptyArray> = Exclude<P, Q[number]> extends never
    ? (Exclude<Q[number], P> extends never ? Q : ReadonlyArray<P>)
    : readonly [...Q, Exclude<P, Q[number]>]
export function assertTypeEquals<P, Q extends NoneEmptyArray>(_test: CompareUnionWithArray<P, Q>): void { }

'/workspaces/tictactoe_spiral/lib_ts/util.ts'

In [6]:
%%node export


export var sleep = (ms: number) => new Promise(resolve => setTimeout(resolve, ms))


'/workspaces/tictactoe_spiral/lib_ts/util.ts'

In [7]:
%%node export

export var timeout = async <T>(prom: Promise<T>, ms: number) => {
	let timer
	try {
        return await Promise.race([
            prom,
            new Promise((_r, rej) => timer = setTimeout(rej, ms, new Error('timeout')))
        ])
    } finally {
        return clearTimeout(timer)
    }
}


'/workspaces/tictactoe_spiral/lib_ts/util.ts'

In [8]:
%%node export

var runId = Math.random().toString(36).substring(2, 5)

var startTimestamp = new Date().getTime()
export var newTimestamp = () => new Date().getTime() - startTimestamp

export var range = (n: number) => Array.from(Array(n).keys())

var pad = (n: string, count: number): string => n.length < count ? pad(`0${n}`, count) : `${n}`

export var env = process.env
export var CODESPACE_NAME = env.CODESPACE_NAME
export var IS_TEST = !!env.IS_TEST

export var getLog =
    (getLocals = () => ({}), argsColor = '#888') =>
        (...args: any[]) => {
            IS_TEST
                ? console.log(
                    `[${pad(newTimestamp().toString(), 6)}] ${runId}`,
                    JSON.stringify(args),
                    JSON.stringify(getLocals())
                )
                : console.log(
                    `[${pad(newTimestamp().toString(), 6)}] ${runId} %c%s %c%s`,
                    `font-weight: bold; color: ${argsColor}`,
                    JSON.stringify(args),
                    'font-weight: bold; color: #444',
                    JSON.stringify(getLocals())
                )
        }

'/workspaces/tictactoe_spiral/lib_ts/util.ts'

In [9]:
%%node run

console.log({ envKeys: Object.keys(env).sort().join('\n') })

['{',
 "  envKeys: 'ABCD\\n' +",
 "    'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL\\n' +",
 "    'BROWSER\\n' +",
 "    'BUN_INSTALL\\n' +",
 "    'CLICOLOR\\n' +",
 "    'CLOUDENV_ENVIRONMENT_ID\\n' +",
 "    'CODESPACES\\n' +",
 "    'CODESPACE_NAME\\n' +",
 "    'CONDA_DIR\\n' +",
 "    'CONDA_SCRIPT\\n' +",
 "    'ContainerVersion\\n' +",
 "    'DEBIAN_FLAVOR\\n' +",
 "    'DOCKER_BUILDKIT\\n' +",
 "    'DOTNET_ROOT\\n' +",
 "    'DOTNET_SKIP_FIRST_TIME_EXPERIENCE\\n' +",
 "    'DYNAMIC_INSTALL_ROOT_DIR\\n' +",
 "    'ELECTRON_RUN_AS_NODE\\n' +",
 "    'ENABLE_DYNAMIC_INSTALL\\n' +",
 "    'GEM_HOME\\n' +",
 "    'GEM_PATH\\n' +",
 "    'GITHUB_API_URL\\n' +",
 "    'GITHUB_CODESPACES_PORT_FORWARDING_DOMAIN\\n' +",
 "    'GITHUB_CODESPACE_TOKEN\\n' +",
 "    'GITHUB_GRAPHQL_URL\\n' +",
 "    'GITHUB_REPOSITORY\\n' +",
 "    'GITHUB_SERVER_URL\\n' +",
 "    'GITHUB_TOKEN\\n' +",
 "    'GITHUB_USER\\n' +",
 "    'GIT_PAGER\\n' +",
 "    'GOPATH\\n' +",
 "    'GOROOT\\n' +",
 "    'GR

# python

In [10]:
#export

import uuid

def new_id():
    return uuid.uuid4().hex

def list_partition(lines, predicate):
    l1, l2 = [], []
    for line in lines:
        (l1, l2)[not predicate(line)].append(line)
    return l1, l2

def write_file(filename, text):
    with open(filename, 'w') as f:
        return f.write(text)

def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()


In [11]:
#export

from time import sleep
from IPython.display import display, Javascript
import subprocess
import os
import uuid
import re

def get_notebook_name():
    if '__vsc_ipynb_file__' in globals():
        ipynb_path = __vsc_ipynb_file__
    else:
        magic = new_id()
        print(f'<magic:{magic}>')

        display(Javascript('IPython.notebook.save_checkpoint();'))

        nb_name = None
        while nb_name is None:
            try:
                sleep(0.1)
                nb_name = subprocess.check_output(f'grep -l {magic} *.ipynb', shell=True).decode().strip()
            except:
                pass

        ipynb_path = os.path.join(os.getcwd(), nb_name)

    ipynb_body = read_file(ipynb_path)
    return re.search(r'\# default_exp (\w+) *', ipynb_body).group(1)

_notebook_name = None


In [12]:
_notebook_name = get_notebook_name()
_notebook_name

'util'

In [13]:
#export

import subprocess

def run_shell(*args, **kwargs):
    try:
        return subprocess.run(*args, shell=True, capture_output=True, **kwargs)
    except Exception as e:
        print(f'nbs/00_util.ipynb() > run_shell () error. args={args} kwargs={kwargs} e={e}')
        raise e
    
def run(*args, **kwargs):
    shell = run_shell(*args, **kwargs)
    stdout = shell.stdout.decode()
    stderr = shell.stderr.decode()
    return_code = shell.returncode
    if return_code != 0:
        print(f'nbs/00_util.ipynb() > run () > error. args="{args}" kwargs="{kwargs}" return_code="{return_code}" stdout="{stdout}" stderr="{stderr}"')
        raise Exception(stderr)
    return stdout.strip('\n')

def run_node(code, timeout=60, *args, **kwargs):
    e = ' --input-type=module -e ' if '\n' in code or '.' not in code else ''
    return run(
        f'''NODE_NO_WARNINGS=1 node --loader ts-node/esm --es-module-specifier-resolution=node {e} \'{code}\'''', 
        *args, 
        timeout=timeout, 
        **kwargs
    )


# py compile

In [14]:
!nbdev_build_lib

/home/codespace/.local/lib/python3.10/site-packages/nbformat/__init__.py:92: DuplicateCellId: Non-unique cell id 'c042685c' detected. Corrected to '0b8da59d'.
  validate(nb)
/home/codespace/.local/lib/python3.10/site-packages/nbformat/__init__.py:92: DuplicateCellId: Non-unique cell id 'c516db25' detected. Corrected to 'dd080f5b'.
  validate(nb)
/home/codespace/.local/lib/python3.10/site-packages/nbformat/__init__.py:92: DuplicateCellId: Non-unique cell id 'a6829b54' detected. Corrected to 'a143c9ef'.
  validate(nb)
Converted 00_components.ipynb.
Converted 00_util.ipynb.
Converted 00_util_node.ipynb.
Converted 01_spiral_api.ipynb.
Converted 02_ipython_magic.ipynb.
/home/codespace/.local/lib/python3.10/site-packages/nbformat/__init__.py:92: DuplicateCellId: Non-unique cell id 'c042685c' detected. Corrected to '46a031df'.
  validate(nb)
/home/codespace/.local/lib/python3.10/site-packages/nbformat/__init__.py:92: DuplicateCellId: Non-unique cell id 'c516db25' detected. Corrected to 'e5f1c